<a href="https://colab.research.google.com/github/miriamvelazque/Aprendizaje_Automatico/blob/main/AA_final_exploracion_preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Exploración y Preprocesamiento del Dataset**

##1. Introducción

#Entrega 2 del proyecto de Aprendizaje Automático."
**Objetivo:** construir un dataset unificado para predecir el riesgo de inundación en barrios populares del Gran Buenos Aires.

Vamos a unir cuatro datasets geoespaciales, construir variables predictoras, y preparar el dataset para modelado supervisado.

##2. Preparación del entorno Carga de Datasets

In [2]:
# Carga manual de archivos
from google.colab import files
uploaded = files.upload()


Saving du_techo_cuerpos_de_agua.geojson to du_techo_cuerpos_de_agua.geojson
Saving du_techo_cursos_de_agua.geojson to du_techo_cursos_de_agua.geojson
Saving du_techo_microbasurales.geojson to du_techo_microbasurales.geojson
Saving du_techo_zonas_inundables.csv to du_techo_zonas_inundables.csv


In [18]:
# Preparamos el entorno
import pandas as pd
import geopandas as gpd

# Cargar los datasets
zonas_inundables = pd.read_csv("du_techo_zonas_inundables.csv")
cuerpos_agua = gpd.read_file("du_techo_cuerpos_de_agua.geojson")
cursos_agua = gpd.read_file("du_techo_cursos_de_agua.geojson")
microbasurales = gpd.read_file("du_techo_microbasurales.geojson")

En esta sección se cargan los cuatro datasets que componen la base de datos del proyecto.

du_techo_zonas_inundables.csv contiene información tabular sobre zonas afectadas por inundaciones, incluyendo la variable objetivo se_inunda_.

du_techo_cuerpos_de_agua.geojson, du_techo_cursos_de_agua.geojson y du_techo_microbasurales.geojson son capas geoespaciales en formato GeoJSON que representan elementos ambientales relevantes. Se utilizan pandas para el archivo CSV y geopandas para los archivos GeoJSON.

Todos los archivos fueron obtenidos de fuentes públicas: DU-Techo y RENABAP (01//10/2025), y se subieron manualmente a Colab para su procesamiento.
Dataset (fuente)

https://www.datos.gob.ar/dataset/habitat-factores-riesgo-barrios-populares-gran-buenos-aires



##3. Exploración Inicial y Codificación de la Variable Objetivo

Vamos a analizar como está estructurado el dataset principal (du_techo_zonas_inundables.csv) y preparar la variable se_inunda_ para el modelo.

In [19]:
# Exploramos la estructura del dataset
zonas_inundables.info() # vemos estructura general
zonas_inundables.head() # vemos primeras filas


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2746 entries, 0 to 2745
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   WKT         2746 non-null   object 
 1   id          2746 non-null   int64  
 2   id_poligon  2741 non-null   float64
 3   se_inunda_  2746 non-null   object 
 4   con_que_fr  2746 non-null   object 
 5   provinicia  2746 non-null   object 
 6   departamen  2746 non-null   object 
 7   localidad   2746 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 171.8+ KB


,WKT,id,id_poligon,se_inunda_,con_que_fr,provinicia,departamen,localidad
0,MULTIPOLYGON (((-58.2143471291028 -26.21995308...,1,17325.0,"SÍ, SÓLO EN UN SECTOR DEL BARRIO",SOLAMENTE CUANDO DILUVIA (UNA O DOS VECES POR ...,Formosa,Formosa,Formosa
1,MULTIPOLYGON (((-58.2179226698168 -26.20925840...,2,17322.0,"SÍ, SÓLO EN UN SECTOR DEL BARRIO",CADA VEZ QUE LLUEVE FUERTE (MUCHAS VECES POR AÑO),Formosa,Formosa,Formosa
2,MULTIPOLYGON (((-58.2334597447517 -26.20322391...,3,18578.0,"SÍ, TODO EL BARRIO",SOLAMENTE CUANDO DILUVIA (UNA O DOS VECES POR ...,Formosa,Formosa,Formosa
3,MULTIPOLYGON (((-58.2764032110572 -26.27741542...,4,12001.0,"SÍ, SÓLO EN UN SECTOR DEL BARRIO",SOLAMENTE CUANDO DILUVIA (UNA O DOS VECES POR ...,Formosa,Formosa,Villa del Carmen
4,MULTIPOLYGON (((-58.2777912542224 -26.27851061...,5,12000.0,"SÍ, SÓLO EN UN SECTOR DEL BARRIO",SOLAMENTE CUANDO DILUVIA (UNA O DOS VECES POR ...,Formosa,Formosa,Villa del Carmen


In [20]:
# Vemos distribución de la variable objetivo
# (incluyendo si hay celdas con valores nulos)
zonas_inundables['se_inunda_'].value_counts(dropna=False)

,count
se_inunda_,
"SÍ, TODO EL BARRIO",1374
"SÍ, SÓLO EN UN SECTOR DEL BARRIO",1372


Se utiliza value_counts(dropna=False) para contar todas las categorías presentes en la variable se_inunda_, incluyendo los valores faltantes (NaN). Esto permite detectar si hay registros incompletos que podrían afectar el modelado o requerir imputación o descarte.

In [21]:
# Inspeccionamos el nombre de todas las columnas del dataset
zonas_inundables.columns


Index(['WKT', 'id', 'id_poligon', 'se_inunda_', 'con_que_fr', 'provinicia',
       'departamen', 'localidad'],
      dtype='object')

Se inspeccionan las columnas del dataset zonas_inundables para identificar las variables disponibles.

Las columnas incluyen:

identificadores (id, id poligon),

información geográfica (provincia, departamen, localidad), y

variables relacionadas con el riesgo de inundación (se_inunda_, con_que_fr).

la columna WKT, contiene geometrías en formato texto

In [8]:
# Exploramos los valores únicos de las columnas categóricas
# Imprime los valores únicos
for col in zonas_inundables.columns:
    if zonas_inundables[col].dtype == 'object':
        print(f"\nValores únicos en {col}:")
        print(zonas_inundables[col].unique())



Valores únicos en WKT:
['MULTIPOLYGON (((-58.2143471291028 -26.2199530868835,-58.2132678708971 -26.2189249124508,-58.2124189527699 -26.2184193481019,-58.21191 -26.218202,-58.210982 -26.218694,-58.2117790166929 -26.2186614916535,-58.212575 -26.219125,-58.2135402582056 -26.2201280771377,-58.2143471291028 -26.2199530868835)))'
 'MULTIPOLYGON (((-58.2179226698168 -26.2092584003946,-58.2200152561458 -26.2037898846083,-58.2194674176553 -26.2038326865085,-58.218947810129 -26.2037211901894,-58.218888 -26.204343,-58.2183626077814 -26.2053971437224,-58.218106 -26.205912,-58.2184509925514 -26.2076616878627,-58.217725 -26.209195,-58.2179226698168 -26.2092584003946)))'
 'MULTIPOLYGON (((-58.2334597447517 -26.2032239116216,-58.2330780486096 -26.2041470664554,-58.234624085 -26.204715812,-58.2355109229999 -26.204996287,-58.2355719719999 -26.2048782,-58.2357920768874 -26.2043956307624,-58.2355450237146 -26.2043008661955,-58.2357106933368 -26.2039012425633,-58.2350769559999 -26.2037040049003,-58.234516

**Interpretación de la exploración inicial**

##Variable se_inunda_
Valores únicos encontrados:

"SÍ, TODO EL BARRIO" (1374 registros)

"SÍ, SÓLO EN UN SECTOR DEL BARRIO" (1372 registros)

No se detectaron registros con la categoría "NO SE INUNDA", lo cual indica que el dataset está compuesto exclusivamente por zonas que presentan algún nivel de inundación.
Es por ello que tomamos la decisión de trabajar con una clasificación binaria supervisada, diferenciando entre inundación total (2) e inundación parcial (1).

##Variable con_que_fr
Valores únicos encontrados:

"CADA VEZ QUE LLUEVE FUERTE (MUCHAS VECES POR AÑO)" (1488)

"OCASIONALMENTE (ALGUNAS VECES POR AÑO)" (638)

"SOLAMENTE CUANDO DILUVIA (UNA O DOS VECES POR AÑO)" (572)

"Otro, especificar" (43)

"NS/NC" (2)

Otros valores con muy baja frecuencia (1–2 registros)

Esta variable aporta información sobre la frecuencia de inundación, lo cual puede ser útil como variable predictora. Se observa una predominancia de zonas que se inundan con frecuencia alta. Podríamos agrupar en categorías (Alta, Media, Baja) para facilitar el modelado.

##Variable provincia, departamento y localidad

Valores únicos encontrados: Más de 20 provincias. Más de 50 departamentos. Más de 100 localidades.
 En los 3 casos, aunque las variables aportan contexto geográfico, se decide no incluirlas como variable predictora debido a que:

Tienen muchas categorías.

La ubicación ya está representada por coordenadas (latitud, longitud).

Podría introducir ruido en el modelo y sobreajuste.

##Variable WKT

Valores únicos: Geometrías en formato MULTIPOLYGON

Esta columna contiene la geometría de cada zona en formato texto. No se usará directamente en el modelo, pero se puede utilizar para calcular variables derivadas como el centroide o el área.

In [10]:
# Vemos la distribución de la variable con_que_fr
zonas_inundables['con_que_fr'].value_counts(dropna=False)

,count
con_que_fr,
CADA VEZ QUE LLUEVE FUERTE (MUCHAS VECES POR AÑO),1488
OCASIONALMENTE (ALGUNAS VECES POR AÑO),638
SOLAMENTE CUANDO DILUVIA (UNA O DOS VECES POR AÑO),572
"Otro, especificar",43
NS/NC,2
CADA VEZ QUE LLUEVE (MUCHAS VECES POR AÑO),2
CADA VEZ QUE LLUEVE FUERTE (MUCHAS VECES AL AÑO),1


##Interpretación

La mayoría de las zonas se inundan frecuentemente: más de la mitad (1488 de 2746) reportan que se inundan cada vez que llueve fuerte.

Hay una proporción significativa que se inunda ocasionalmente (638) o solo en eventos extremos (572).

Los valores "Otro, especificar" y "NS/NC" son pocos, pero podrían representar respuestas abiertas o falta de información.

Aparecen duplicados con pequeñas variaciones en la redacción, como "CADA VEZ QUE LLUEVE FUERTE (MUCHAS VECES POR AÑO)" y "CADA VEZ QUE LLUEVE FUERTE (MUCHAS VECES AL AÑO)", vamos a normalizar estos textos.

In [24]:
# Preprocesamiento (agrupar en tres categorías simplificadas)
# Transformación de variable categórica textual (con_que_fr) en una versión más simplificada y estructurada
def codificar_frecuencia(valor):
    valor = str(valor).upper()
    if "CADA VEZ" in valor or "MUCHAS VECES" in valor:
        return "Alta"
    elif "OCASIONALMENTE" in valor:
        return "Media"
    elif "SOLAMENTE" in valor or "UNA O DOS VECES" in valor:
        return "Baja"
    else:
        return "Otro"

zonas_inundables['frecuencia_codificada'] = zonas_inundables['con_que_fr'].apply(codificar_frecuencia)

zonas_inundables['frecuencia_codificada'].value_counts(dropna=False)

,count
frecuencia_codificada,
Alta,1491
Media,638
Baja,572
Otro,45


Se agrupan los valores en cuatro categorías:

Alta: zonas que se inundan cada vez que llueve fuerte o muchas veces por año.

Media: zonas que se inundan ocasionalmente.

Baja: zonas que solo se inundan en eventos extremos.

Otro: respuestas abiertas o no clasificables.

Esta transformación permite reducir la variabilidad textual y facilita el uso de la variable como predictora en el modelo supervisado.

##**Resultados de la exploración:**
Tras analizar las columnas se_inunda_ y con_que_fr, se observa que el dataset no contiene registros con la categoría "NO SE INUNDA". Esto indica que el archivo du_techo_zonas_inundables.csv está compuesto exclusivamente por zonas afectadas por inundaciones, ya sea total o parcialmente. Por lo tanto, se ajusta la variable objetivo a una clasificación binaria:

1: Inundación parcial

2: Inundación total

Esta decisión permite construir un modelo supervisado que prediga el grado de afectación en zonas vulnerables, cumpliendo con el objetivo del proyecto.

Además, se analiza la variable con_que_fr, que describe la frecuencia con la que se inunda cada zona. Para facilitar su uso como variable predictora, se realiza un preprocesamiento que agrupa sus valores textuales en cuatro categorías simplificadas:

Alta: Zonas que se inundan cada vez que llueve fuerte o muchas veces por año

Media: Zonas que se inundan ocasionalmente

Baja: Zonas que solo se inundan en eventos extremos

Otro: Respuestas abiertas o no clasificables

Esta nueva variable, frecuencia_codificada, se conservará como predictora en el modelo, ya que aporta información complementaria al tipo de inundación.

##4. Construcción de Variables Predictoras Geoespaciales

Incorporamos variables espaciales que puedan influir en el riesgo de inundación, como la proximidad a cuerpos de agua y a cursos de agua (distancias) o la presencia de microbasurales (cantidad). Estas variables se extraen a partir de las capas geoespaciales disponibles en formato GeoJSON.

**Justificación del uso de geopandas y shapely**
Para enriquecer el dataset con variables ambientales, se utilizan capas geoespaciales que contienen información sobre cuerpos de agua, cursos de agua y microbasurales. Estas capas están en formato GeoJSON, lo que permite trabajar con geometrías mediante la librería geopandas, que extiende pandas para datos espaciales. Además, se utiliza shapely, una librería especializada en operaciones geométricas, como el cálculo de distancias entre polígonos y puntos. El uso de estas herramientas está justificado por la necesidad de generar variables predictoras basadas en la ubicación y el entorno físico de cada zona.